In [1]:
!pip install pygame
!pip install numpy

In [1]:
import sys  
import os        
import pygame
import numpy as np
import math
import time
import random


pygame.init()
BG_COLOR = (0, 0, 0)
screen = pygame.display.set_mode((600, 500), pygame.RESIZABLE)
SCREEN_SIZE = screen.get_size()


class Game:
    def __init__(self, screen):
        self.screen = screen
        self.starship = Starship()
        self.bullets = []
        self.asteroids = []
        self.mouse_pressed = False
        self.fire_rate = 0.2 # 1 / 5
        
    def handle_events(self, frame):
        for event in pygame.event.get():     
            if event.type == pygame.QUIT:  
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                self.mouse_pressed = True
            if event.type == pygame.MOUSEBUTTONUP:
                self.mouse_pressed = False
        if self.mouse_pressed == True and (time.time() - self.starship.last_bullet_time) > self.fire_rate:
            new_bullet = self.starship.fire()
            self.bullets.append(new_bullet)
        if frame % 100 == 0:
            self.asteroids.append(Asteroid())
                
    def move_objects(self, objects_list):
        for obj_idx, _ in enumerate(objects_list):
            for o_idx, _ in enumerate(objects_list[obj_idx]):
                objects_list[obj_idx][o_idx].move()
                
    def draw(self, objects_list):
        self.screen.fill(BG_COLOR)
        for objects in objects_list:
            for obj in objects:
                self.screen.blit(obj.image, obj.rect)
        pygame.display.update()
        
    def check_collisions(self):
        asteroids_rects = []
        for asteroid in self.asteroids:
            asteroids_rects.append(asteroid.rect)
            
        for idx, bullet in enumerate(self.bullets):
            if bullet.pos[0] > SCREEN_SIZE[0] or bullet.pos[1] > SCREEN_SIZE[1] or bullet.pos[0] < 0 or bullet.pos[1] < 0:
                del self.bullets[idx]
                
        for idx, bullet in enumerate(self.bullets):
            hit = bullet.rect.collidelist(asteroids_rects)   # -1 если столкновений нет
            if hit != -1:
                self.asteroids += self.asteroids[hit].explode()
                del self.asteroids[hit]
                del asteroids_rects[hit]
                del self.bullets[idx]
                
        hit = self.starship.rect.collidelist(asteroids_rects)
        if hit != -1:
            sys.exit()
                
        
                
    def run(self):
        frame = 0
        clock = pygame.time.Clock()
        while True:
            clock.tick(60)
            self.handle_events(frame)
            self.check_collisions()
            self.move_objects([[self.starship], self.bullets, self.asteroids])
            self.draw([[self.starship], self.bullets, self.asteroids])
            frame += 1
        
class Starship:
    def __init__(self):
        self.pos = np.array([SCREEN_SIZE[0] / 2, SCREEN_SIZE[1] / 2])
        self.original_image = pygame.image.load(os.path.join("images", "starship.png"))
        self.image = self.original_image
        self.rect = starship_rect = self.image.get_rect(center=self.pos)
        self.last_bullet_time = 0
        
    def move(self):
        mouse_pos = pygame.mouse.get_pos()   # 300 200
        direction = mouse_pos - self.pos
        angle = self.calculate_angle(mouse_pos)
        self.speed = direction / 30
        self.pos += self.speed                # 250 250  + 4 5 = 254 255
        self.image = pygame.transform.rotate(self.original_image, int(angle))
        self.rect = self.image.get_rect(center=self.pos)
        
    def calculate_angle(self, mouse_pos):
        rel_x, rel_y = mouse_pos - self.pos    
        angle = (180 / math.pi) * -math.atan2(rel_y, rel_x) + 90
        return angle
    
    def fire(self):
        self.last_bullet_time = time.time()
        return Bullet(self.pos.copy())
    
    
class Bullet:
    original_image = pygame.image.load(os.path.join("images", "bullet.png"))
    def __init__(self, pos):
        self.pos = pos
        mouse_pos = pygame.mouse.get_pos()
        self.direction = mouse_pos - self.pos
        self.speed = self.direction / max(abs(self.direction)) * 10 
        self.angle = self.calculate_angle(mouse_pos)
        self.image = pygame.transform.rotate(self.original_image, int(self.angle))
        self.rect = self.image.get_rect(center=self.pos)
        
    def move(self):
        self.pos += self.speed
        self.rect = self.image.get_rect(center=self.pos)
    
        
    def calculate_angle(self, mouse_pos):
        rel_x, rel_y = mouse_pos - self.pos    
        angle = (180 / math.pi) * -math.atan2(rel_y, rel_x) + 90
        return angle
        
class Asteroid:
    ast_variants = [("small", pygame.image.load(os.path.join("images", "ast1_small.png")), (10, 10)),
                   ("small", pygame.image.load(os.path.join("images", "ast2_small.png")), (10, 10)),
                   ("small", pygame.image.load(os.path.join("images", "ast3_small.png")), (10, 10)),
                   ("small", pygame.image.load(os.path.join("images", "ast4_small.png")), (10, 10)),
                   ("medium", pygame.image.load(os.path.join("images", "ast1_medium.png")), (40, 40)),
                   ("medium", pygame.image.load(os.path.join("images", "ast2_medium.png")), (40, 40)),
                   ("medium", pygame.image.load(os.path.join("images", "ast3_medium.png")), (40, 40)),
                   ("medium", pygame.image.load(os.path.join("images", "ast4_medium.png")), (40, 40)),
                   ("large", pygame.image.load(os.path.join("images", "ast1_large.png")), (105, 105)),
                   ("large", pygame.image.load(os.path.join("images", "ast2_large.png")), (105, 105)),
                   ("large", pygame.image.load(os.path.join("images", "ast3_large.png")), (105, 105)),
                   ("large", pygame.image.load(os.path.join("images", "ast4_large.png")), (105, 105))]
    """
    original_images = [pygame.image.load(os.path.join("images", "ast1_medium.png")),
                       pygame.image.load(os.path.join("images", "ast2_medium.png")),
                       pygame.image.load(os.path.join("images", "ast3_medium.png")),
                       pygame.image.load(os.path.join("images", "ast4_medium.png"))]
    """
    def __init__(self, pos=None, speed=None, ast_type=None):
        if (pos is None) and (speed is None) and (ast_type is None):
            self.init_rand_asteroid()
        else:
            self.init_asteroid_fragment(pos, speed, ast_type)
        
        
    def init_rand_asteroid(self):
        self.type, self.original_image, hitbox_shape = random.choice(self.ast_variants)
        self.image = self.original_image
        self.w, self.h = hitbox_shape
        self.pos = random.choice([self.left_pos, self.right_pos, self.top_pos, self.bottom_pos])()
        self.rect = self.image.get_rect(center=self.pos)
        self.direction = pygame.mouse.get_pos() - self.pos
        self.speed = self.direction / 300
        
    def init_asteroid_fragment(self, pos, speed, ast_type):
        ast_type_variants = list(filter(lambda x: x[0] == ast_type, self.ast_variants))
        self.type, self.original_image, hitbox_shape = random.choice(ast_type_variants)
        self.pos = pos
        self.image = self.original_image
        self.rect = self.image.get_rect(center=self.pos)
        self.speed = speed
        
        
    def move(self):
        self.check_borders()
        self.pos += self.speed
        self.rect = self.image.get_rect(center=self.pos)
        
    def left_pos(self):
        return np.array([-150, random.uniform(0, SCREEN_SIZE[1])])
    
    def right_pos(self):
        return np.array([SCREEN_SIZE[0] + 150, random.uniform(0, SCREEN_SIZE[1])])
    
    def top_pos(self):
        return np.array([random.uniform(0, SCREEN_SIZE[0]), -150])
    
    def bottom_pos(self):
        return np.array([random.uniform(0, SCREEN_SIZE[0]), SCREEN_SIZE[1] + 150])
    
    def check_borders(self):
        if self.pos[0] > (SCREEN_SIZE[0] + 150):
            self.pos[0] = -150
        elif (self.pos[0] + 150) < 0:
            self.pos[0] = SCREEN_SIZE[0] + 150
        if (self.pos[1] - 150) > SCREEN_SIZE[1]:
            self.pos[1] = -150
        elif (self.pos[1] + 150) < 0:
            self.pos[1] = SCREEN_SIZE[1] + 150
        # self.rect = self.image.get_rect(center=self.pos)
        
    def explode(self):
        fragments = []
        if self.type == "large":
            fragments += [Asteroid(self.pos_offset(), self.speed_offset(), "medium") for _ in range(2)]
        if self.type == "medium":
            fragments += [Asteroid(self.pos_offset(), self.speed_offset(), "small") for _ in range(3)]
        if self.type == "small":
            pass
        return fragments
    
    def speed_offset(self):
        offset = self.speed * 0.5 + np.random.uniform(-0.5, 0.5, 2)
        return offset
    
    def pos_offset(self):
        offset = self.pos + np.random.uniform(-10, 10, 2)
        return offset
        
        
game = Game(screen)
game.run()

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

/home/roman/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
asteroids_rects = []
        for asteroid in self.asteroids:
            asteroid_rects.append(asteroid.rect)

In [ ]:
asteroids_rects = [asteroid.rect for asteroid in self.asteroids]

In [1]:
b = [number**2 for number in range(6)]

In [2]:
b

[0, 1, 4, 9, 16, 25]

In [9]:
def is_even(x):
    return x % 2 == 0

In [10]:
list(filter(is_even, [1, 2, 3, 4, 5]))

[2, 4]